In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_covid()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=.6)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (361, 80)
19.75% missing data
Class distribution: (array([0, 1]), array([195, 166]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': [80, 80],
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 4001, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 0.68 sec - Loss 0.082385 - ACC 70.08% - ACC Mean 70.08% - AUC 73.56% - AUC Mean 73.56% - Deter 000
Ite 00050 - 0.12 sec - Loss 0.079130 - ACC 67.28% - ACC Mean 70.07% - AUC 71.65% - AUC Mean 74.10% - Deter 028
Ite 00100 - 0.43 sec - Loss 0.072999 - ACC 69.38% - ACC Mean 68.83% - AUC 73.61% - AUC Mean 73.30% - Deter 078
Ite 00150 - 0.11 sec - Loss 0.068739 - ACC 70.65% - ACC Mean 69.61% - AUC 74.37% - AUC Mean 74.59% - Deter 128
Ite 00200 - 0.42 sec - Loss 0.067652 - ACC 68.65% - ACC Mean 69.47% - AUC 72.67% - AUC Mean 74.27% - Deter 178
Ite 00250 - 0.13 sec - Loss 0.067117 - ACC 69.81% - ACC Mean 68.53% - AUC 72.41% - AUC Mean 73.57% - Deter 228
Ite 00300 - 0.12 sec - Loss 0.066545 - ACC 69.04% - ACC Mean 68.93% - AUC 75.17% - AUC Mean 73.86% - Deter 278
Ite 00350 - 0.50 sec - Loss 0.065885 - ACC 68.31% - ACC Mean 68.96% - AUC 74.58% - AUC Mean 74.00% - Deter 328
Ite 00400 - 0.13 sec - Loss 0.065170 - ACC 67.98% - ACC Mean 68.53% - AUC 73.99% - AUC Mean 73.66% - Deter 378
I

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 69.96% +- 0.0% - auc: 73.72% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 57.06% +- 7.37% - auc: 61.94% +- 7.91%
SFIL RUN 2/10 - acc: 55.48% +- 7.67% - auc: 65.37% +- 8.99%
SFIL RUN 3/10 - acc: 61.29% +- 6.61% - auc: 66.93% +- 3.11%
SFIL RUN 4/10 - acc: 61.35% +- 6.38% - auc: 70.99% +- 6.31%
SFIL RUN 5/10 - acc: 60.91% +- 5.88% - auc: 64.84% +- 6.14%
SFIL RUN 6/10 - acc: 62.81% +- 6.56% - auc: 68.26% +- 6.5%
SFIL RUN 7/10 - acc: 62.12% +- 6.38% - auc: 67.16% +- 9.21%
SFIL RUN 8/10 - acc: 59.99% +- 6.45% - auc: 70.59% +- 5.93%
SFIL RUN 9/10 - acc: 59.08% +- 6.49% - auc: 66.08% +- 9.95%
SFIL RUN 10/10 - acc: 60.03% +- 6.57% - auc: 69.56% +- 6.51%
SFIL GLOBAL - acc: 60.01% +- 2.16% - auc: 67.17% +- 2.65%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 69.78% +- 4.28% - auc: 74.29% +- 6.15%
PFIL RUN 2/10 - acc: 69.66% +- 6.96% - auc: 72.96% +- 7.39%
PFIL RUN 3/10 - acc: 64.92% +- 3.83% - auc: 68.91% +- 3.61%
PFIL RUN 4/10 - acc: 66.84% +- 2.79% - auc: 71.24% +- 4.13%
PFIL RUN 5/10 - acc: 68.23% +- 2.77% - auc: 74.13% +- 3.06%
PFIL RUN 6/10 - acc: 64.08% +- 4.89% - auc: 70.05% +- 3.41%
PFIL RUN 7/10 - acc: 72.4% +- 2.6% - auc: 74.26% +- 3.17%
PFIL RUN 8/10 - acc: 67.59% +- 2.45% - auc: 70.27% +- 4.42%
PFIL RUN 9/10 - acc: 67.34% +- 5.47% - auc: 73.36% +- 4.02%
PFIL RUN 10/10 - acc: 64.54% +- 5.48% - auc: 72.53% +- 7.13%
PFIL GLOBAL - acc: 67.54% +- 2.49% - auc: 72.2% +- 1.86%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 59.88% +- 4.28% - auc: 63.82% +- 4.47%
SPOL RUN 2/10 - acc: 53.09% +- 6.75% - auc: 58.96% +- 7.76%
SPOL RUN 3/10 - acc: 67.62% +- 8.57% - auc: 68.69% +- 7.28%
SPOL RUN 4/10 - acc: 57.51% +- 6.17% - auc: 60.07% +- 7.53%
SPOL RUN 5/10 - acc: 60.96% +- 4.45% - auc: 65.1% +- 7.16%
SPOL RUN 6/10 - acc: 59.47% +- 6.33% - auc: 62.02% +- 5.54%
SPOL RUN 7/10 - acc: 62.91% +- 4.82% - auc: 67.32% +- 4.35%
SPOL RUN 8/10 - acc: 62.24% +- 3.07% - auc: 67.49% +- 3.92%
SPOL RUN 9/10 - acc: 60.36% +- 3.43% - auc: 62.67% +- 4.58%
SPOL RUN 10/10 - acc: 61.23% +- 8.28% - auc: 62.71% +- 7.77%
SPOL GLOBAL - acc: 60.53% +- 3.54% - auc: 63.89% +- 3.07%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 65.96% +- 3.12% - auc: 70.4% +- 4.13%
PPOL RUN 2/10 - acc: 70.83% +- 5.97% - auc: 73.4% +- 6.47%
PPOL RUN 3/10 - acc: 72.76% +- 3.08% - auc: 76.11% +- 3.9%
PPOL RUN 4/10 - acc: 64.73% +- 5.18% - auc: 69.88% +- 5.54%
PPOL RUN 5/10 - acc: 66.45% +- 6.24% - auc: 70.47% +- 6.49%
PPOL RUN 6/10 - acc: 62.27% +- 2.57% - auc: 67.59% +- 4.5%
PPOL RUN 7/10 - acc: 65.9% +- 7.34% - auc: 70.6% +- 5.04%
PPOL RUN 8/10 - acc: 64.94% +- 5.56% - auc: 69.84% +- 7.39%
PPOL RUN 9/10 - acc: 65.17% +- 3.88% - auc: 69.54% +- 5.22%
PPOL RUN 10/10 - acc: 70.65% +- 4.43% - auc: 73.08% +- 5.86%
PPOL GLOBAL - acc: 66.97% +- 3.15% - auc: 71.09% +- 2.31%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
